In [1]:
import pandas as pd
from datasets import load_dataset , list_datasets
datasets = load_dataset('stochastic/random_streetview_images_pano_v0.0.2')
datasets

Found cached dataset parquet (/Users/ramamurthi/.cache/huggingface/datasets/stochastic___parquet/stochastic--random_streetview_images_pano_v0.0.2-65d25597b51c2d04/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'country_iso_alpha2', 'latitude', 'longitude', 'address'],
        num_rows: 11054
    })
})

In [1]:
from geoguessr.preprocess import prepare_data

train_dl , test_dl , train_data_size , test_data_size  , geoguessr_dataset = prepare_data()


Found cached dataset parquet (/Users/ramamurthi/.cache/huggingface/datasets/stochastic___parquet/stochastic--random_streetview_images_pano_v0.0.2-65d25597b51c2d04/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
from fastai.callback.progress import ProgressCallback
from fastai.vision.data import DataLoaders
from fastai.learner import Learner
from fastai.optimizer import OptimWrapper

dls = DataLoaders(train_dl , test_dl)
dls


In [13]:
from fastai.vision.all import *

In [15]:
learn = vision_learner(dls, 'resnet26d', metrics=error_rate, path='.').to_fp16()

AssertionError: `n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`

In [16]:
??vision_learer

Object `vision_learer` not found.


In [18]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 2.7 MB/s eta 0:00:00m eta 0:00:010:01:01
You should consider upgrading via the '/Users/ramamurthi/pet_projects/geoguessr/env/bin/python3.8 -m pip install --upgrade pip' command.


In [23]:
import timm
pretrained_resnet_18 = timm.create_model('resnet18', pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/ramamurthi/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [24]:
pretrained_resnet_18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

In [26]:
avail_pretrained_models = timm.list_models(pretrained=True)
avail_pretrained_models

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_rw_224',
 'coatnet_1_rw_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_rw_224',
 'coatnet_rmlp_1_rw_224',
 'coatnet_rmlp_2_rw_224',
 'coatnet_rmlp_nano_rw_224',
 'coatnext_nano_rw_224',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_atto',
 'convnext_atto_